# Final Bootes master catalogue

This notebook presents the combination of the HELP standard catalogue with the aperture matched photometry from Brown which gives superior photometric redhsifts.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
708e28f (Tue May 8 18:05:21 2018 +0100)
This notebook was executed on: 
2018-05-16 18:12:48.187415


In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [29]:
brown_catalogue_name ="master_catalogue_bootes_brown_20180517.fits"
help_catalogue_name ="master_catalogue_bootes_help_20180517.fits"

brown_cross_id_name = "master_list_cross_ident_bootes_brown_20180517.fits"
help_cross_id_name = "master_list_cross_ident_bootes_help_20180517.fits"

In [ ]:
brown_catalogue = Table.read("{}/{}".format(OUT_DIR, brown_catalogue_name))
help_catalogue = Table.read("{}/{}".format(OUT_DIR, help_catalogue_name))

In [3]:
brown_cross_id = Table.read("{}/{}".format(OUT_DIR,brown_cross_id_name))['full_help_id', 'help_id']
brown_cross_id['help_id'].name = 'brown_help_id'
brown_cross_id['full_help_id'].name = 'help_help_id'

In [ ]:
in_brown = set(brown_catalogue.colnames)
in_help = set(help_catalogue.colnames)
in_both = in_help.intersection(in_brown)
only_in_brown = in_brown - in_help

for col in brown_catalogue.colnames:
    brown_catalogue[col].name = "brown_{}".format(col)
    
for col in help_catalogue.colnames:
    help_catalogue[col].name = "help_{}".format(col)

## 2 - Join the two catalogues taking brown if both are available

In [ ]:
master_catalogue = join(brown_cross_id, brown_catalogue, join_type='left')
master_catalogue = join(master_catalogue, help_catalogue, join_type='left')
             



In [ ]:
for col in in_both:
    has_brown = (master_catalogue['brown_help_id'] != '-1')
                
    try:
        has_brown = has_brown  & ~np.isnan(master_catalogue["brown_{}".format(col)])
    except TypeError:
        print("brown_{} is not a column of floats".format(col))
    master_catalogue["help_{}".format(col)][has_brown] = master_catalogue["brown_{}".format(col)][has_brown]
    master_catalogue["help_{}".format(col)].name = col
    master_catalogue.remove_column("brown_{}".format(col))
    
for col in only_in_brown:
    master_catalogue["brown_{}".format(col)].name = col

In [ ]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

## 3  - Crop to HELP area

The Bootes Brown catalogue slightly goes outside the HELP Bootes field.

In [ ]:
help_bootes_moc = MOC(filename="../../dmu2/dmu2_field_coverages/Bootes_MOC.fits")
in_HELP_mask = inMoc(
    master_catalogue['ra'], master_catalogue['brown_dec'],
    help_bootes_moc 
)

master_catalogue = master_catalogue[in_HELP_mask]

## 4 - Check and save

In [ ]:
master_catalogue[:10].show_in_notebook()

In [ ]:
master_catalogue = master_catalogue.filled()
master_catalogue.write("{}/master_catalogue_bootes{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## 5 - Make final cross ID catalogue

In [33]:
brown_cross_id = Table.read("{}/{}".format(OUT_DIR,brown_cross_id_name))
brown_cross_id['help_id'].name = 'brown_help_id'
brown_cross_id['full_help_id'].name = 'help_help_id'


help_cross_id = Table.read("{}/{}".format(OUT_DIR,help_cross_id_name))
help_cross_id.remove_columns(['specz_id', 'sdss_id'])
help_cross_id['help_id'].name = 'help_help_id'

final_cross_id = join(brown_cross_id, help_cross_id, join_type='left')





for col in final_cross_id.colnames:
    final_cross_id[col].fill_value = -1
final_cross_id = final_cross_id.filled()

final_cross_id['specz_id'][final_cross_id['specz_id'] == ''] = '-1'

final_cross_id.add_column(Column(data=final_cross_id['help_help_id'], name = 'help_id'))
have_brown_id = (final_cross_id['brown_help_id'] != '-1')
final_cross_id['help_id'][have_brown_id] = final_cross_id['brown_help_id'][have_brown_id]

In [34]:
final_cross_id.colnames

['brown_i_id',
 'brown_irac_id',
 'help_help_id',
 'brown_help_id',
 'specz_id',
 'sdss_id',
 'ps1_id',
 'legacy_id',
 'ibis_id',
 'ndwfs_id',
 'uhs_id',
 'zbootes_id',
 'datafusion_intid',
 'help_id']

In [35]:
columns = ['help_id',
           'brown_help_id',
           'help_help_id',
           'brown_i_id',
           'brown_irac_id',
           'specz_id',
           'sdss_id',
           'ps1_id',
           'legacy_id',
           'ibis_id',
           'ndwfs_id',
           'uhs_id',
           'zbootes_id',
           'datafusion_intid']
print(set(columns) - set(final_cross_id.colnames))
final_cross_id = final_cross_id[columns]

set()


In [36]:
final_cross_id[:10].show_in_notebook()

idx,help_id,brown_help_id,help_help_id,brown_i_id,brown_irac_id,specz_id,sdss_id,ps1_id,legacy_id,ibis_id,ndwfs_id,uhs_id,zbootes_id,datafusion_intid
0,HELP_J142500.285+331035.838,HELP_J142500.285+331035.838,-1,394263,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,HELP_J143318.051+332158.118,HELP_J143318.051+332158.118,-1,473154,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,HELP_J142939.235+354635.105,HELP_J142939.235+354635.105,-1,2019839,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,HELP_J142446.142+331756.854,HELP_J142446.142+331756.854,-1,445407,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,HELP_J142639.609+354724.470,HELP_J142639.609+354724.470,-1,2027735,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,HELP_J142446.456+324136.432,HELP_J142446.456+324136.432,-1,161462,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,HELP_J142440.580+354503.991,HELP_J142440.580+354503.991,-1,2005436,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,HELP_J142742.433+325234.332,HELP_J142742.433+325234.332,-1,242716,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,HELP_J142510.784+344929.132,HELP_J142510.784+344929.132,-1,1446199,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,HELP_J143221.937+354511.185,HELP_J143221.937+354511.185,-1,2006597,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [13]:

final_cross_id.write("master_list_cross_ident_bootes{}".format(SUFFIX), overwrite=True)

False
False
False
False
False
False
